# Cuckoo search implementation
#### By Wei Wang and Pragnesh Krishnan Ramachandran Vidyasagar

In [12]:
import pandas as pd
import numpy as np

In [13]:
## --------------- initialize cuckoo search params ---------
def initialize_cs_params():
    # Discovery rate of alien eggs/solutions
    n=25
    pa=0.25
    nest = []
    ## Change this if you want to get better results
    # Tolerance
    Tol=1.0e-5
    ## Simple bounds of the search domain
    # Lower bounds
    nd=15 
    Lb=-5000.0 * np.ones(nd);
    # Upper bounds
    Ub=5000.0 * np.ones(nd);
    nest = np.zeros((n,nd))
    # Random initial solutions
    for i in range(n):
        arr = np.array(Ub * np.random.rand(nd))
        nest[i,:] = arr.copy()
    return n, pa, Tol, nd, Lb, Ub, nest

In [14]:
## You can replace the following by your own functions
# A d-dimensional objective function
def fobj(u):
## d-dimensional sphere function sum_j=1^d (u_j-1)^2. 
#  with a minimum at (1,1, ...., 1); 
    #z=sum((u-1).^2);
    return sum([i**2 for i in u])

In [15]:
# Application of simple constraints
def simplebounds(s,Lb,Ub):
  # Apply the lower bound
    ns_tmp=np.array([j if i < j else i for i,j in zip(s,Lb) ])
    s = ns_tmp.copy()
  # Apply the upper bound
    ns_tmp=np.array([j if i > j else i for i,j in zip(s,Ub) ])
    s = ns_tmp.copy()
    return s

In [16]:
## Replace some nests by constructing new solutions/nests
def empty_nests(nest,Lb,Ub,pa):
    # A fraction of worse nests are discovered with a probability pa
    nrows = nest.shape[0]
    ncols = nest.shape[1]
    # Discovered or not -- a status vector
    # if pa=0.25, then atleast 73% of values will have 1
    K=np.random.rand(nrows, ncols) > pa

    # In the real world, if a cuckoo's egg is very similar to a host's eggs, then 
    # this cuckoo's egg is less likely to be discovered, thus the fitness should 
    # be related to the difference in solutions.  Therefore, it is a good idea 
    # to do a random walk in a biased way with some random step sizes.  
    ## New solution by biased/selective random walks
    #stepsize = rand * (nest(randperm(n),:) - nest(randperm(n),:));
    stepsize = np.random.rand() * (nest[np.random.permutation(nrows),:] - \
                                   nest[np.random.permutation(nrows),:]) 
    
    #new_nest = nest + stepsize.*K;
    new_nest = nest + stepsize * K
    for j in range(nrows):
        s=new_nest[j,:].copy()
        new_nest[j,:]=simplebounds(s,Lb,Ub)
    
    return new_nest


In [17]:
## Find the current best nest
#function [fmin,best,nest,fitness]=get_best_nest(nest,newnest,fitness)
def get_best_nest(nest, newnest, fitness):
# Evaluating all new solutions
    for j in range(nest.shape[0]):
        fnew=fobj(newnest[j,:])
        if fnew<=fitness[j]:
            fitness[j]=fnew.copy()
            nest[j,:]=newnest[j,:].copy()
    # Find the current best
    fmin, K = min(fitness), fitness.argmin()
    best=nest[K,:].copy()
    return fmin, best, nest, fitness

In [18]:
## Get cuckoos by ramdom walk
#function nest=get_cuckoos(nest,best,Lb,Ub)
def get_cuckoos(nest,best,Lb,Ub):
    # Levy flights
    #n=size(nest,1);
    n =nest.shape[0] 
    # Levy exponent and coefficient
    # For details, see equation (2.21), Page 16 (chapter 2) of the book
    # X. S. Yang, Nature-Inspired Metaheuristic Algorithms, 2nd Edition, Luniver Press, (2010).
    beta=3./2.;
    sigma=(mt.gamma(1+beta)* \
           np.sin(np.pi*beta/2)/(mt.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta);

    for j in range(n):
        s=nest[j,:].copy()
        # This is a simple way of implementing Levy flights
        # For standard random walks, use step=1;
        ## Levy flights by Mantegna's algorithm
        u=np.random.randn(len(s))*sigma;
        v=np.random.randn(len(s));
        step=u/abs(v)**(1/beta);

        # In the next equation, the difference factor (s-best) means that 
        # when the solution is the best solution, it remains unchanged.     
        stepsize=0.01*step*(s-best);
        # Here the factor 0.01 comes from the fact that L/100 should the typical
        # step size of walks/flights where L is the typical lenghtscale; 
        # otherwise, Levy flights may become too aggresive/efficient, 
        # which makes new solutions (even) jump out side of the design domain 
        # (and thus wasting evaluations).
        # Now the actual random walks or flights
        s=s+stepsize*np.random.randn(len(s));
        # Apply simple bounds/limits
        nest[j,:]=simplebounds(s,Lb,Ub);


    return nest

In [19]:
n, pa, Tol, nd, Lb, Ub, nest = None, None, None, None, None, None, None
n, pa, Tol, nd, Lb, Ub, nest = initialize_cs_params()

In [20]:
# Get the current best
fitness=10**10*np.ones(n);
[fmin,bestnest,nest,fitness]=get_best_nest(nest,nest,fitness);
N_iter=0;
## Starting iterations
while fmin > Tol:
    # Generate new solutions (but keep the current best)
    new_nest=get_cuckoos(nest,bestnest,Lb,Ub); 
    [fnew,best,nest,fitness]=get_best_nest(nest,new_nest,fitness);
    # Update the counter
    N_iter=N_iter+n; 
    # Discovery and randomization
    new_nest=empty_nests(nest,Lb,Ub,pa) ;
    
    # Evaluate this set of solutions
    [fnew,best,nest,fitness]=get_best_nest(nest,new_nest,fitness);
    # Update the counter again
    N_iter=N_iter+n;
    # Find the best objective so far  
    if fnew < fmin:
        fmin = fnew;
        bestnest = best;    
## End of iterations


In [21]:
bestnest

array([ 2.55306125e-04,  8.33490727e-04, -2.67375016e-04,  1.11358207e-04,
       -6.40226396e-04,  2.19533483e-04,  3.11275569e-05,  2.45896733e-04,
        6.69277173e-04, -5.62982815e-05, -3.74875361e-04,  2.60244322e-03,
       -2.68093223e-04, -8.71002406e-04,  1.53305340e-04])

In [22]:
new_nest

array([[-5.42947346e-04,  1.90999547e-03,  3.22087380e-04,
        -4.91800270e-04,  1.77857172e-04, -2.20887516e-03,
        -1.73920929e-05, -7.64486060e-04,  1.71228988e-03,
        -1.84981555e-03,  2.28621498e-04,  1.66322429e-03,
         6.60596960e-04,  1.72408937e-03, -9.28328884e-04],
       [ 1.96214454e-04,  1.33172463e-02, -2.57128582e-03,
        -2.03574684e-03, -6.53201288e-03, -1.46755775e-03,
        -4.97827951e-03,  3.97927920e-04, -4.35488552e-03,
         5.88651529e-03,  9.13162845e-04,  1.10674998e-03,
         7.66909806e-04,  5.28501928e-03,  8.04905656e-02],
       [-1.40152898e-03,  1.16845217e-03, -7.29346395e-05,
         1.06911575e-03, -7.49463555e-05, -2.19035148e-03,
         1.30549860e-04, -5.31388503e-04,  2.00454304e-03,
         3.07261738e-04,  2.86548737e-04,  1.35366716e-03,
        -1.32904442e-03,  1.78243835e-03, -1.14799607e-03],
       [-1.96228907e-03,  9.22202047e-04,  4.31646055e-03,
        -1.66443948e-03,  2.52189166e-03, -4.21941952

In [137]:
# Observe all eggs in all nests, and look at the magnitude.
# Majority should be small 10^-3
nest

array([[-1.59568069e-04, -2.06768779e-03, -8.61061297e-04,
         6.57142890e-04,  1.30588675e-03,  7.30481321e-04,
        -2.30023021e-03,  1.40149655e-04,  8.21507949e-04,
         1.28500571e-03,  2.14762106e-03,  1.27422755e-04,
        -6.84900023e-04, -1.14435901e-04, -1.36615636e-03],
       [-8.45702435e-04, -1.59500983e-03,  9.38743499e-04,
         8.75161089e-05, -3.63309065e-04, -1.01365268e-03,
        -8.44266057e-04,  8.54977839e-04, -1.50263427e-03,
        -2.54821059e-04,  2.52111883e-03,  1.62881369e-03,
        -6.93830712e-04,  6.36025127e-04, -7.85267682e-04],
       [ 9.03189919e-02, -2.74300157e+00,  1.90355253e+00,
         1.10657217e-01, -1.09890824e+01,  1.69420844e-01,
        -7.81927126e+01,  3.03674816e-01, -9.36086876e-01,
        -7.67916049e-01, -9.33076830e-01, -8.98552320e-03,
        -2.45763708e+00,  4.63250882e-01,  4.05043469e-01],
       [-7.45682261e-04, -9.47678697e-04, -1.77478165e-03,
        -1.18307875e-03,  1.38497146e-04, -9.47246164